# Seção 1: Inicialização e informações da GPU

In [1]:
!pip install git+git://github.com/canesche/nvcc4jupyter.git
!git clone https://github.com/canesche/nvcc4jupyter
%load_ext nvcc_plugin

  Cloning git://github.com/canesche/nvcc4jupyter.git to /tmp/pip-req-build-q13dr341
  Running command git clone -q git://github.com/canesche/nvcc4jupyter.git /tmp/pip-req-build-q13dr341
  Created wheel for ColabPlugin: filename=ColabPlugin-blind-py3-none-any.whl size=12727 sha256=543f6a8e9681b2a68559dd0615d102b17365526c4ff6f2c9d975fa399d83692e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ee8q0_ps/wheels/97/a0/61/b9e5e1f61c5cfd624df291d57f6731a0f5832606b9ced448ef
Failed to build ColabPlugin
    Running setup.py install for ColabPlugin ... done
  DEPRECATION: ColabPlugin was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
Cloning into 'nvcc4jupyter'...
remote: Enumerating objects: 1147, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (271/271),

In [2]:
%%gpu
#include <cuda.h>
#include <stdio.h>

int main(){
    int dev = 0;
    cudaDeviceProp deviceProp;
    cudaGetDeviceProperties(&deviceProp, dev);

    printf("Using Device %d:         %s\n",     dev, deviceProp.name);
    printf("Total global memory:     %u B\n",   deviceProp.totalGlobalMem);
    printf("Threads per block:       %d\n",     deviceProp.maxThreadsPerBlock);
    printf("Maximum Grid Size:       %d,%d,%d\n",deviceProp.maxGridSize[0],
                                                deviceProp.maxGridSize[1],
                                                deviceProp.maxGridSize[2]);
    printf("Maximum block size:      %d,%d,%d\n",deviceProp.maxThreadsDim[0],
                                                deviceProp.maxThreadsDim[1],
                                                deviceProp.maxThreadsDim[2]);
    return 0;
}

Using Device 0:         Tesla K80
Total global memory:     3407020032 B
Threads per block:       1024
Maximum Grid Size:       2147483647,65535,65535
Maximum block size:      1024,1024,64



# Seção 2: Atividade prática

No jogo campo minado, diversas bombas são distribuídas sobre um campo representado por uma matriz, e cada célula sem bomba guarda o número de bombas existentes nas 8 células ao seu redor. Dessa forma, o campo abaixo à esquerda,
onde cada bomba é representada por um *, pode ser armazenado pela matriz de inteiros mostrada à direita, onde cada bomba é representada por um 9. 
```
1 * 2 2 *                1 9 2 2 9 
2 4 * 3 1                2 4 9 3 1
* 5 * 2                  9 5 9 2 0
* * 3 1                  9 9 3 1 0
3 * 2                    3 9 2 0 0
```
Faça um programa que distribui de forma aleatória um número de bombas e, em seguida, utiliza a GPU para calcular de forma paralela o número de bombas próximas a cada célula sem bomba.

Depois que estiver certo que seu kernel funciona, implemente também uma versão sequencial para fazer o mesma cálculo e insira marcadores de tempo para calcular e exibir o tempo gasto em cada uma das duas abordagens. Aumente o tamanho do campo progressivamente para valores muito grandes (ex: 10000x10000) e perceba que o tempo de cálculo na versão paralela tem um aumento relativamente baixo em relação à versão sequencial.

In [75]:
%%nvprof

#include <stdio.h>
#include <cuda.h>
#include <stdexcept>
#include <assert.h>
#include <unistd.h>
#include <chrono>
using namespace std;

#define BLOCK_SIZE 32
//#define DEBUG

/* A função CHECK a seguir pode ser utilizada caso seja necessário identificar
erros em chamadas da biblioteca CUDA. Exemplo: 
CHECK(cudaMalloc((void**)&x,size));
*/
#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
    }                                                                          \
}

void distribuiBombas(int8_t *a, int linhas, int colunas, int num){
    if (num > linhas*colunas)
        throw std::invalid_argument("Número de bombas extrapolou o máximo possível.");

    for (int i=0; i<linhas; ++i)
        for (int j=0; j<colunas; ++j)
            a[i*colunas+j]=0;

    int lin, col;
    for (int i=0; i<num; ++i) {
        lin=rand()%linhas;
        col=rand()%colunas;
        while (a[lin*colunas+col] == 9) {
            lin=rand()%linhas;
            col=rand()%colunas;
        }
        a[lin*colunas+col]=9;
    }
}


void imprimeMatriz(int8_t *a, int linhas, int colunas ){
    int v;
    for (int i=0; i<linhas; ++i) {
        for (int j=0; j<colunas; ++j) {
            v=a[i*colunas+j];
            if (v==9) printf("* ");
            //else if (v==0) printf("  ");
            //else printf("%d ",v);
            else printf("%d ",v);
        }
        printf("\n");
    }
    printf("\n");
}


__global__ void geraVizinhos(int8_t *a, int8_t *b, int linhas, int colunas){
    int i = blockIdx.y*blockDim.y + threadIdx.y;
    int j = blockIdx.x*blockDim.x + threadIdx.x;
    int pos = i*colunas+j;

    if (a[pos] == 9) b[pos] = 9;

    //testa se a posicao é valida, podem haver threads tentando acessar posições fora da matriz
    if(pos > linhas*colunas || i>= linhas || j>= colunas || a[pos] == 9) return;

    //lista de vizinhos, como todas as posições a serem testadas
    int vizinhos[8][2] = {{0,1},{1,1}, {-1,1}, {-1,0}, {1,0}, {0,-1},{1,-1}, {-1,-1}};
    int auxI = 0, auxJ = 0;

    //cada thread testa seus 8 vizinhos
    for(int k = 0; k<8; k++){

        //move para o vizinho
        auxI = i+vizinhos[k][0];
        auxJ = j+vizinhos[k][1];

      if(auxI >= 0 and auxI< linhas && auxJ >= 0 && auxJ < colunas && a[auxI*colunas+auxJ] == 9)
        ++b[pos];
    }
}

//mesma lógica da gpu, porém sequencial
void vizinhosSeq(int8_t* m, int linhas, int colunas) {
  int vizinhos[][2] = {{0,1},{1,1}, {-1,1}, {-1,0}, {1,0}, {0,-1},{1,-1}, {-1,-1}};
  int auxX = 0, auxY = 0;

  for(int i = 0; i < linhas; i++){   
    for(int j = 0; j < colunas; j++){
        if(m[i*colunas+j] == 9) continue;

        for(int k = 0; k < 8; k++) {
          auxX = i+vizinhos[k][0];
          auxY = j+vizinhos[k][1];

          if(auxX >=0 && auxX < linhas && auxY >=0 && auxY < colunas && m[auxX*colunas + auxY] == 9)
            m[i*colunas + j] ++;
        }
    }
  }
}

//inicializa uma matriz com todas as posições zeradas
void inicializa(int8_t *a, int linhas, int colunas){
    for (int i=0; i<linhas; ++i)
        for (int j=0; j<colunas; ++j)
            a[i*colunas+j]=0;
}


//confere se duas matrizes são iguais
void confere(int8_t *a, int8_t *b, int linhas, int colunas){
    for (int i=0; i<linhas; ++i)
        for (int j=0; j<colunas; ++j)
            if(a[i*colunas+j]!=b[i*colunas+j]){ printf("Erro nas posições %d %d\n",i,j); return;}
}

int main() {
    //não estava conseguindo medir o tempo com valores mais altos, dava erro no buffer size ??
    int linhas = 8000, colunas=8000; //64*10^6 posições
    int numBombas = 40000000;         //4*10^6 bombas => uma a cada 8 posições tinha bomba 
    int8_t *h_A, *h_B;
    int8_t *d_A, *d_B;


    int size = linhas*colunas*sizeof(int8_t);
    h_A = (int8_t*)malloc(size);
    h_B = (int8_t*)malloc(size);

    inicializa(h_A,linhas,colunas);
    inicializa(h_B,linhas,colunas);



    distribuiBombas(h_A,linhas,colunas,numBombas);

    cudaMalloc((void **) &d_A, size);
    cudaMalloc((void **) &d_B, size);


    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);

    dim3 grid (ceil(colunas*1.0/BLOCK_SIZE),ceil(linhas*1.0/BLOCK_SIZE),1);
    dim3 block (BLOCK_SIZE,BLOCK_SIZE,1);
    geraVizinhos<<<grid,block>>>(d_A,d_B,linhas,colunas);
    


    //cudaDeviceReset();

    cudaMemcpy(h_B, d_B, size, cudaMemcpyDeviceToHost);
    
    //printf("%d \n",h_A[0]);

    auto start = chrono::system_clock::now();
    vizinhosSeq(h_A,linhas,colunas);
    auto end = chrono::system_clock::now();
    chrono::duration<double> time = end - start;

    #ifdef DEBUG
    {
        imprimeMatriz(h_A,linhas,colunas);
        imprimeMatriz(h_B,linhas,colunas);
    
        confere(h_B,h_A,linhas,colunas);
    }
    #endif
    
    printf("A execução na CPU levou %fs.\n",time.count());



    free(h_A);
    free(h_B);
    cudaFree(d_A);
    cudaFree(d_B);

    


    return 0;    
}



==3477== NVPROF is profiling process 3477, command: /content/code.out
==3477== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
A execução na CPU levou 2.676481s.
==3477== Profiling application: /content/code.out
==3477== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   60.64%  27.081ms         1  27.081ms  27.081ms  27.081ms  geraVizinhos(char*, char*, int, int)
                   20.48%  9.1466ms         1  9.1466ms  9.1466ms  9.1466ms  [CUDA memcpy HtoD]
                   18.88%  8.4291ms         1  8.4291ms  8.4291ms  8.4291ms  [CUDA memcpy DtoH]
      API calls:   78.14%  193.59ms         2  96.797ms  223.59us  193.37ms  cudaMalloc
                   18.15%  44.960ms         2  22.480ms  9.2606ms  35.699ms  cudaMemcpy
                    3.40%  8.4275ms         2  4.2137ms  258.91us  8.1686ms  cudaFree
                    0.21%  513.20us         1  513.20us  513.20us  513.20us  cu